In [4]:
from typing import List
import numpy as np
import pandas as pd
from lightning.pytorch.utilities.types import TRAIN_DATALOADERS, EVAL_DATALOADERS
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch import nn
import torch
import torch.nn.functional as F
from pytorch_forecasting import TimeSeriesDataSet
from lightning.pytorch.callbacks import EarlyStopping
import lightning.pytorch as pl

In [9]:
df = pd.read_csv(
    "../data/dnaq_15Min_interval.csv",
    parse_dates=["collection_time"],
    index_col="collection_time",
)
N_LAGS = 7
HORIZON = 1
TARGET = ["Ia", "Ib", "Ic", "Ua", "Ub", "Uc"]
n_vars = df.shape[1]

In [10]:
class MultivariateSeriesDataModule(pl.LightningDataModule):
    def __init__(
            self,
            data: pd.DataFrame,
            target_variables: List[str],
            n_lags: int,
            horizon: int,
            test_size: float = 0.2,
            batch_size: int = 32,
    ):
        super().__init__()
        self.data = data
        self.batch_size = batch_size
        self.test_size = test_size
        self.n_lags = n_lags
        self.horizon = horizon
        self.target_variables = target_variables
        self.target_scaler = {k: MinMaxScaler() for k in target_variables}
        self.feature_names = [
            col for col in data.columns if col not in self.target_variables
        ]

        self.training = None
        self.validation = None
        self.test = None
        self.predict_set = None

        self.setup()

    def preprocess_data(self):
        self.data["time_index"] = np.arange(len(self.data))
        self.data["group_id"] = 0

    def split_data(self):
        time_indices = self.data["time_index"].values
        train_indices, test_indices = train_test_split(
            time_indices, test_size=self.test_size, shuffle=False
        )
        train_indices, val_indices = train_test_split(
            train_indices, test_size=0.1, shuffle=False
        )
        return train_indices, val_indices, test_indices

    def scale_target(self, df, indices):
        for c in self.target_variables:
            scaled_values = self.target_scaler[c].transform(df.loc[indices, [c]])
            df.loc[indices, c] = scaled_values

    def setup(self, stage=None):
        self.preprocess_data()
        train_indices, val_indices, test_indices = self.split_data()

        train_df = self.data.loc[self.data["time_index"].isin(train_indices)]
        val_df = self.data.loc[self.data["time_index"].isin(val_indices)]
        test_df = self.data.loc[self.data["time_index"].isin(test_indices)]

        for c in self.target_variables:
            self.target_scaler[c].fit(train_df[[c]])

        self.scale_target(train_df, train_df.index)
        self.scale_target(val_df, val_df.index)
        self.scale_target(test_df, test_df.index)

        self.training = TimeSeriesDataSet(
            train_df,
            time_idx="time_index",
            target=self.target_variables,
            group_ids=["group_id"],
            max_encoder_length=self.n_lags,
            max_prediction_length=self.horizon,
            time_varying_unknown_reals=self.feature_names + self.target_variables,
            scalers={name: MinMaxScaler() for name in self.feature_names},
        )
        self.validation = TimeSeriesDataSet.from_dataset(self.training, val_df)
        self.test = TimeSeriesDataSet.from_dataset(self.training, test_df)
        self.predict_set = TimeSeriesDataSet.from_dataset(
            self.training, self.data, predict=True, stop_randomization=True
        )
        
    def train_dataloader(self) -> TRAIN_DATALOADERS:
        return self.training.to_dataloader(batch_size=self.batch_size, shuffle=False)
    
    def val_dataloader(self) -> EVAL_DATALOADERS:
        return self.validation.to_dataloader(batch_size=self.batch_size, shuffle=False)
    
    def test_dataloader(self) -> EVAL_DATALOADERS:
        return self.test.to_dataloader(batch_size=self.batch_size, shuffle=False)
    
    def predict_dataloader(self) -> EVAL_DATALOADERS:
        return self.predict_set.to_dataloader(batch_size=1, shuffle=False)

In [11]:
class MultiOutputLSTM(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, num_layers, horizon, n_output):
        super().__init__()
        self.n_output = n_output
        self.horizon = horizon
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.output_dim = int(self.n_output * self.horizon)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, self.output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.lstm.num_layers, x.size(0), self.hidden_dim).to(
            self.device
        )
        c0 = torch.zeros(self.lstm.num_layers, x.size(0), self.hidden_dim).to(
            self.device
        )
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x["encoder_cont"])

        y_pred = y_pred.unsqueeze(-1).view(-1, self.horizon, self.n_output)
        y_pred = [y_pred[:, :, i] for i in range(self.n_output)]

        loss = [F.mse_loss(y_pred[i], y[0][i]) for i in range(self.n_output)]

        loss = torch.mean(torch.stack(loss))

        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x["encoder_cont"])

        y_pred = y_pred.unsqueeze(-1).view(-1, self.horizon, self.n_output)
        y_pred = [y_pred[:, :, i] for i in range(self.n_output)]

        loss = [F.mse_loss(y_pred[i], y[0][i]) for i in range(self.n_output)]

        loss = torch.mean(torch.stack(loss))

        self.log("val_loss", loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self(x["encoder_cont"])

        y_pred = y_pred.unsqueeze(-1).view(-1, self.horizon, self.n_output)
        y_pred = [y_pred[:, :, i] for i in range(self.n_output)]

        loss = [F.mse_loss(y_pred[i], y[0][i]) for i in range(self.n_output)]

        loss = torch.mean(torch.stack(loss))

        self.log("test_loss", loss)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch

        y_pred = self(x["encoder_cont"])

        y_pred = y_pred.unsqueeze(-1).view(-1, self.horizon, self.n_output)
        y_pred = [y_pred[:, :, i] for i in range(self.n_output)]

        return y_pred

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

In [12]:
model = MultiOutputLSTM(
    input_dim=n_vars, hidden_dim=32, num_layers=2, horizon=HORIZON, n_output=len(TARGET)
)

In [13]:
print(model)

MultiOutputLSTM(
  (lstm): LSTM(9, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=6, bias=True)
)


In [14]:
datamodule = MultivariateSeriesDataModule(
    data=df,
    n_lags=N_LAGS,
    horizon=HORIZON,
    test_size=0.2,
    batch_size=32,
    target_variables=TARGET,
)

In [15]:
early_stop_callback = EarlyStopping(
    monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min"
)

In [16]:
from lightning.pytorch.loggers import TensorBoardLogger

logger = TensorBoardLogger('lightning_logs', name="low-power-equipment-prediction")
trainer = pl.Trainer(max_epochs=30, callbacks=[early_stop_callback], logger=logger)
trainer.fit(model, datamodule)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type   | Params
--------------------------------
0 | lstm | LSTM   | 14.0 K
1 | fc   | Linear | 198   
--------------------------------
14.2 K    Trainable params
0         Non-trainable params
14.2 K    Total params
0.057     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/raopend/Workspace/stock-price-prediction-system/notebooks/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/raopend/Workspace/stock-price-prediction-system/notebooks/.venv/lib/python3.10/site-packages/lightning/pytorch/utilities/data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 32. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
/Users/raopend/Workspace/stock-price-prediction-system/notebooks/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoad

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=30` reached.


In [17]:
trainer.test(model, datamodule.test_dataloader())

/Users/raopend/Workspace/stock-price-prediction-system/notebooks/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss          0.007004112470895052
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.007004112470895052}]

In [21]:
forecasts = trainer.predict(model=model, datamodule=datamodule)

/Users/raopend/Workspace/stock-price-prediction-system/notebooks/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [22]:
forecasts

[[tensor([[0.0980]]),
  tensor([[0.1690]]),
  tensor([[0.0897]]),
  tensor([[0.7159]]),
  tensor([[0.7116]]),
  tensor([[0.7289]])]]

In [25]:
model.eval()
torch.onnx.export(
    model,
    torch.randn(1, N_LAGS, n_vars),
    "../model/multi_output_lstm.onnx",
    export_params=True,
)

/Users/raopend/Workspace/stock-price-prediction-system/notebooks/.venv/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:4662: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(
